In [1]:
from mip import *
import networkx as nx
import pickle
from itertools import product
import numpy as np
import pandas as pd
import time

In [15]:
# The network
g = nx.read_gml('networks/usa_99.gml', label="id")
L = len(g.edges)
links = range(L)


# The cut SRLGs
with open ('min_cut_SRLGs/usa_99_10-4', 'rb') as fp:
    cut_srlgs = pickle.load(fp)
S = len(cut_srlgs)


# The matrix of the intensity values, dimensions: [L,P,M] (link, position, magnitude)
intensity = np.load('intensities/usa_99_ds23.npy')


# The matrix of earthquake probabilities, dimensions: [P,M] (position, magnitude)
prob_matrix = pd.read_csv('earthquake_probabilities/usa_ds23.csv').drop(['Lat', 'Long'], axis=1).to_numpy()
P, M = prob_matrix.shape
epicenters = range(P)
magnitudes = range(M)


# Parameters
Hnull = 6
T = 0.0001
cost = 1

In [ ]:
# Compressing the problem, to 1 SLRG and the minimum number of earthquakes
cut_srlgs = cut_srlgs[:30]
S = len(cut_srlgs)
print(cut_srlgs)

column_mask = np.full(intensity.shape[2], fill_value=False, dtype=bool)
row_mask = np.full(intensity.shape[1], fill_value=False, dtype=bool)

for srlg in cut_srlgs:
    srlg_mask = np.full(intensity.shape[1:], fill_value=True, dtype=bool)
    for link in srlg:
        idx = list(g.edges).index(link)
        print(idx, link, g.edges[link]['length'])
        link_mask = intensity[idx]>6
        srlg_mask &= link_mask
    column_mask |= srlg_mask.any(axis=0)
    row_mask |= srlg_mask.any(axis=1)

intensity = intensity[:,:,column_mask][:,row_mask]
prob_matrix = prob_matrix[:,column_mask][row_mask]

P, M = prob_matrix.shape
epicenters = range(P)
magnitudes = range(M)

print(f'The shape of the intensity matrix: {intensity.shape}')

In [19]:
srlg = cut_srlgs[2]
edges = list(g.edges)
link_mask = np.full(intensity.shape[1:], fill_value=True, dtype=bool)
for l in srlg:
    l_idx = edges.index(l)
    link_mask &= intensity[l_idx] > H[l_idx]


3718

In [16]:
# Heuristic 1 & 2

def get_SRLG_probability_matrix(srlg, network, intensity_matrix, intensity_tolerance, probability_matrix):
    edges = list(network.edges)
    srlg_occur = np.full(intensity_matrix.shape[1:], fill_value=True, dtype=bool)
    for l in srlg:
        l_idx = edges.index(l)
        srlg_occur &= intensity[l_idx] > intensity_tolerance[l_idx]
    return probability_matrix[srlg_occur]

def remove_improbable_SRLGs(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold):
    active_srlgs = srlgs.copy()
    for s in active_srlgs.copy():
        p = get_SRLG_probability_matrix(s, g, intensity, H, prob_matrix).sum()
        if p < threshold:
            active_srlgs.remove(s)
    return active_srlgs

def countSRLGlinks(srlgs, network):
    edges = list(network.edges)
    partofSRLG = np.zeros(len(edges), dtype=int)
    for srlg in srlgs:
        for l in srlg:
            partofSRLG[edges.index(l)] += 1
    #return dict(zip(edges, partofSRLG))
    return partofSRLG

def get_edge_to_improve_1(srlgs, network):
    edges = list(network.edges)
    partofSRLG = countSRLGlinks(srlgs, network)
    max_indexes = [i for i, j in enumerate(partofSRLG) if j == max(partofSRLG)]
    #print(max_indexes)
    if len(max_indexes) > 1:
        max_index = max_indexes[0]
        min_length = network.edges[edges[max_index]]['length']
        for idx in max_indexes:
            length = network.edges[edges[idx]]['length']
            if length < min_length:
                max_index = idx
                min_length = length
        return max_index
    else:
        return max_indexes[0]

def heuristic_1(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold):
    active_srlgs = remove_improbable_SRLGs(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
    cost = 0
    edges = list(network.edges)
    print(len(active_srlgs))
    while len(active_srlgs):
        edge_to_improve = get_edge_to_improve_1(active_srlgs, network)
        cost += network.edges[edges[edge_to_improve]]['length']
        #print(edge_to_improve)
        intensity_tolerance[edge_to_improve] += 1
        active_srlgs = remove_improbable_SRLGs(active_srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
    print(f'H1 Cost: {cost:.0f}')
    return intensity_tolerance, cost

def get_SRLG_probability_reduction(link_idx, srlg, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold):
    srlg_prob_matrix = get_SRLG_probability_matrix(srlg, network, intensity_matrix, intensity_tolerance, probability_matrix)
    srlg_prob = srlg_prob_matrix.sum()
    intensity_tolerance[link_idx] += 1
    new_srlg_prob_matrix= get_SRLG_probability_matrix(srlg, network, intensity_matrix, intensity_tolerance, probability_matrix)
    new_srlg_prob = new_srlg_prob_matrix.sum()
    intensity_tolerance[link_idx] -= 1
    return srlg_prob - new_srlg_prob

def get_edge_to_improve_2(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold):
    edges = list(network.edges)
    probability_reduction_values = np.zeros(len(edges))
    for idx, edge in enumerate(edges):
        probability_reduction = 0
        for srlg in srlgs:
            if edge in srlg:
                probability_reduction += get_SRLG_probability_reduction(idx, srlg, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
        probability_reduction_values[idx] = probability_reduction / network.edges[edge]['length']
    return np.argmax(probability_reduction_values)

def heuristic_2(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold):
    active_srlgs = remove_improbable_SRLGs(srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
    cost = 0
    edges = list(network.edges)
    print(len(active_srlgs))
    while len(active_srlgs):
        edge_to_improve = get_edge_to_improve_2(active_srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
        cost += network.edges[edges[edge_to_improve]]['length']
        #print(edge_to_improve)
        intensity_tolerance[edge_to_improve] += 1
        active_srlgs = remove_improbable_SRLGs(active_srlgs, network, intensity_matrix, intensity_tolerance, probability_matrix, threshold)
    print(f'H2 Cost: {cost:.0f}')
    return intensity_tolerance, cost

In [ ]:
data = {
    'T': [],
    'Active SRLGs': [],
    'Cost H1': [],
    'Cost H2': [],
    'Cost ILP': [],
    'Runtime H1': [],
    'Runtime H2': [],
    'Runtime ILP': [],
}

upgraded_edges = {}

for idx,T in enumerate(np.concatenate((np.arange(0.01, 0.001, -0.001), np.arange(0.001, 0.0004, -0.0001)))):
    print('\n================')
    print(f'T: {T:.4f}')
    data['T'].append(T)
    
    H = np.ones(L) * 6
    for idx, e in enumerate(g.edges):
        if g.edges[e]['onspine']:
            H[idx] += 0
    active_srlgs = remove_improbable_SRLGs(cut_srlgs, g, intensity, H, prob_matrix, T)
    print(f'Number of active SRLGs: {len(active_srlgs)}')
    data['Active SRLGs'].append(len(active_srlgs))
    
    start = time.perf_counter()
    H = np.ones(L) * 6
    for idx, e in enumerate(g.edges):
        if g.edges[e]['onspine']:
            H[idx] += 0
    H1, cost_1 = heuristic_1(cut_srlgs, g, intensity, H, prob_matrix, T)
    runtime_1 = time.perf_counter() - start
    data['Cost H1'].append(cost_1)
    data['Runtime H1'].append(runtime_1)
    
    start = time.perf_counter()
    H = np.ones(L) * 6
    for idx, e in enumerate(g.edges):
        if g.edges[e]['onspine']:
            H[idx] += 0
    H2, cost_2 = heuristic_2(cut_srlgs, g, intensity, H, prob_matrix, T)
    runtime_2 = time.perf_counter() - start
    data['Cost H2'].append(cost_2)
    data['Runtime H2'].append(runtime_2)
    
    data['Cost ILP'].append(np.nan)
    data['Runtime ILP'].append(np.nan)
    
    upgrade_1, upgrade_2 = [],[]
    for l,e in enumerate(g.edges):
        if H1[l] > 6:
            upgrade_1.append((e,int(H1[l]-6)))
        if H2[l] > 6:
            upgrade_2.append((e,int(H2[l]-6)))
    
    upgraded_edges[idx] = {}
    upgraded_edges[idx]['Upgrade H1'] = upgrade_1
    upgraded_edges[idx]['Upgrade H2'] = upgrade_2

    print('\n================')


In [20]:
pd.DataFrame(data)

,T,Active SRLGs,Cost H1,Cost H2,Cost ILP,Runtime H1,Runtime H2,Runtime ILP
0,0.0100,3,2016.152018,1471.174551,NaN,0.017185,0.041056,NaN
1,0.0090,3,2016.152018,2016.152018,NaN,0.011481,0.031578,NaN
2,0.0080,3,2016.152018,2016.152018,NaN,0.010778,0.038467,NaN
3,0.0070,3,2181.929680,2016.152018,NaN,0.020379,0.059040,NaN
4,0.0060,3,2181.929680,2016.152018,NaN,0.020744,0.039013,NaN
5,0.0050,4,2561.129485,2561.129485,NaN,0.014980,0.049136,NaN
6,0.0040,4,4032.304037,4032.304037,NaN,0.018544,0.061477,NaN
7,0.0030,4,4198.081698,4198.081698,NaN,0.018892,0.062748,NaN
8,0.0020,5,4996.771291,4624.023929,NaN,0.025079,0.131700,NaN
9,0.0010,5,7339.675345,6686.918372,NaN,0.039931,0.200116,NaN


In [21]:
pd.DataFrame(data).to_csv('results/Heuristic_comparison_usa_99_nospine.csv', index=False, float_format='%.4f')
with open('results/Heuristic_upgraded_edges_usa_99_nospine', 'wb') as fp:
    pickle.dump(upgraded_edges, fp)

In [4]:
# ILP

start = time.perf_counter()

#Model
model = Model(sense=MINIMIZE, solver_name=GRB)

#Variables
deltaH = [model.add_var(var_type=INTEGER, lb=0, ub=6) for l,_ in enumerate(g.edges)]
Z = [[[model.add_var(var_type=BINARY) for k in magnitudes] for j in epicenters] for i in range(S)]
Y = [[[model.add_var(var_type=BINARY) for k in magnitudes] for j in epicenters] for i in links]
print("%.1f s:\tVáltozók létrehozva..."%(time.perf_counter()-start))


#Objective Function
model.objective = xsum( g.edges[link_id]['length'] * deltaH[link_idx] for link_idx,link_id in enumerate(g.edges) )
print("%.1f s:\tCélfüggvény létrehozva..."%(time.perf_counter()-start))


#Constraint 1
for l,p,m in product(*[links, epicenters, magnitudes]):
    model.add_constr( Y[l][p][m] >= 1 - ((Hnull + deltaH[l]) / intensity[l,p,m]) )
print("%.1f s:\tElső egyenlet létrehozva..."%(time.perf_counter()-start))


#Constraint 2
for (c,s), p, m in product(*[enumerate(cut_srlgs), epicenters, magnitudes]):
    model.add_constr( Z[c][p][m] >= (xsum(Y[list(g.edges).index(linkID)][p][m] for linkID in s) - len(s) + 1) )
print("%.1f s:\tMásodik egyenlet létrehozva..."%(time.perf_counter()-start))

#Constraint 3
for c,_ in enumerate(cut_srlgs):
    model.add_constr(xsum( Z[c][p][m] * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ) <= T, "c3_"+str(c))
print("%.1f s:\tHarmadik egyenlet létrehozva..."%(time.perf_counter()-start))

#Start optimization
model.optimize()#max_seconds=
print("%.1f s:\tMegoldás megtalálva..."%(time.perf_counter()-start))

gurobi version 9.0 found


44.8 s:	Változók létrehozva...
45.6 s:	Célfüggvény létrehozva...
176.2 s:	Első egyenlet létrehozva...
836.4 s:	Második egyenlet létrehozva...
853.5 s:	Harmadik egyenlet létrehozva...
911.8 s:	Megoldás megtalálva...


In [5]:
# The probability of occurance of the SRLGs
for c,s in enumerate(cut_srlgs):
    print(s, sum(Z[c][p][m].x * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ))

{(6, 9, 0), (8, 16, 0), (8, 15, 0)} 0.000761214297066
{(15, 22, 0), (8, 16, 0)} 0.0005675366584531701
{(6, 9, 0), (0, 7, 0), (6, 20, 0), (7, 8, 0)} 0.0006059366979880004
{(15, 22, 0), (16, 22, 0), (17, 22, 0)} 0.0003968406877110002
{(16, 17, 0), (17, 22, 0)} 0.0009184504510770004
{(6, 9, 0), (6, 20, 0), (6, 7, 0)} 0.0009979877808720006
{(4, 5, 0), (3, 5, 0), (5, 19, 0)} 0.0009638781175647996
{(0, 7, 0), (7, 8, 0), (8, 15, 0), (8, 16, 0), (6, 20, 0)} 0.000935171801873
{(0, 7, 0), (7, 8, 0), (6, 7, 0)} 0.0009807425495840004
{(16, 17, 0), (8, 16, 0), (16, 22, 0)} 0.0009975387505163967
{(8, 16, 0), (8, 15, 0), (6, 20, 0), (6, 7, 0)} 0.00035866550810400005
{(6, 9, 0), (9, 10, 0), (9, 12, 0)} 0.0008132968968906003
{(8, 16, 0), (0, 7, 0), (7, 8, 0), (6, 7, 0)} 0.000494733278545
{(4, 5, 0), (4, 18, 0)} 0.000921298243624
{(20, 21, 0), (6, 20, 0), (0, 20, 0)} 0.00046536536652836986
{(1, 8, 0), (20, 21, 0), (0, 7, 0), (0, 20, 0)} 0.0003866668314809999
{(1, 8, 0), (6, 9, 0), (7, 8, 0)} 0.000804912

In [6]:
# The result
selected = [(deltaH[l].x,e, g.edges[e]['length']) for l,e in enumerate(g.edges) if deltaH[l].x >= 0.5]
print(*selected, sep='\n')
#print([dH.x for dH in deltaH])

(2.0, (0, 20, 0), 66.21522968576761)
(2.0, (4, 5, 0), 99.71896170046314)
(2.0, (6, 7, 0), 80.3366106193597)
(1.0, (6, 20, 0), 236.6408237880778)
(2.0, (7, 8, 0), 79.86433362181208)
(1.0, (8, 16, 0), 143.724658712267)
(1.0, (9, 12, 0), 145.25795577084781)
(2.0, (17, 22, 0), 107.9290298827612)


In [7]:
# Add additional constraint
# It does not deletes Constraint 3 but completes it
for c,s in enumerate(cut_srlgs):
    model.add_constr(xsum( Z[c][p][m] * prob_matrix[p,m] for p,m in product(epicenters,magnitudes) ) <= 0.00001, "c3_"+str(c))

In [8]:
# Start optimization
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

# ---------------------------------------------------------------

In [7]:
def ff(a,b):
    return a*2 + b**2

In [14]:
#koltseg =  [11, 3,3, 6, 1, 9, 16,78]
koltseg = 8
fejlesztes = [17,6,4,61,16,42,156, 7]

y = [1,1,0,1,0,1,0,0]
z = [0,1,1,1,0,1,0,1]

items = len(fejlesztes)

m = Model("fradir")

#decision variable
x = [m.add_var(var_type=BINARY) for i in range(items)]
fejlesztes_Dvar = [m.add_var(var_type=INTEGER, lb=1, ub=6) for i in range(items)]

#objective
m.objective = maximize(xsum(koltseg * fejlesztes[i] * x[i] for i in range(items)))

#cons
#m += xsum(y[i] * x[i] for i in range(items)) >= 2

#m += xsum(z[i] * x[i] for i in range(items)) >= 2

#m += xsum(((y[i] + z[i]) % 2) * x[i]  for i in range(items)) == 0

for i in range(items):
    m += 10 * x[i] <= fejlesztes[i]

#m += xsum(( fejlesztes[i] * x[i]  for i in range(items))) <= 14

m += xsum(x[i] for i in range(items)) == 2


m.optimize()

selected = [i for i in range(items) if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [3, 6]


In [14]:
ff(3,4)

22

# GEKKO

In [6]:
import random
from gekko import GEKKO
import numpy as np

In [7]:
koltseg =  [11, 3,3, 6, 1, 9, 16,78]
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,1,0]
z = [0,1,1,1,0,1,1,1]

prob = [0.5, 1.2, 1.5, 1.7, 0.1, 1.8, 0.4, 1.9]

items = len(koltseg)

# Create model
m = GEKKO()

# Variables
x = m.Array(m.Var,items,lb=0,ub=1,integer=True)
fejlesztes_Dvar = m.Array(m.Var, items, lb=1,ub=6, integer=True)
b = m.Array(m.Var, items, lb=1,ub=8, integer=True)
#x2 = m.Array(m.Var, len(w),lb=0,ub=1,integer = True)

# Objective
m.Maximize(sum(koltseg[i] * fejlesztes_Dvar[i] * x[i] for i in range(items) ))

# Constraint
m.Equation(sum([(y[i] + z[i]) * x[i] for i in range(items)]) == 2)

m.Equation(sum([x[i] for i in range(items)]) == 1)

m.Equation(sum((z[i] * prob[i]) *x[i] for i in range(items)) <=1.6 )

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

# Print the value of the variables at the optimum
print(x)
for i in range(len(x)):
    if x[i][0] == 1.0:
        print("Sorszam: {}., koltseg: {}, fejlesztes: {}".format(i,koltseg[i], fejlesztes_Dvar[i][0]))

[[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [1.0] [0.0]]
Sorszam: 6., koltseg: 16, fejlesztes: 6.0


In [2]:
data = np.load('italy.npy')

In [13]:
pd.DataFrame(data[0]).describe()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
count,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,...,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000,48442.000000
mean,1.154713,1.174508,1.196344,1.220365,1.246715,1.275495,1.306710,1.340423,1.376788,1.415944,...,3.194826,3.325022,3.459114,3.597036,3.738632,3.883800,4.032280,4.183553,4.337154,4.492752
std,0.578387,0.617518,0.658248,0.700550,0.744388,0.789728,0.836545,0.884773,0.934307,0.985046,...,2.112041,2.145883,2.176857,2.204735,2.229386,2.250615,2.268428,2.283199,2.295290,2.304934
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.260501,1.422601,1.584701,1.746801,1.908901,2.071001,2.233101,2.395201,2.557301,2.719401
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.691067,2.853167,3.015267,3.177367,3.339467,3.501567,3.663667,3.825767,3.987867,4.149967
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.004125,...,4.570325,4.732425,4.894525,5.056625,5.218725,5.380825,5.542925,5.705025,5.867125,6.029225
max,6.113586,6.275686,6.437786,6.599886,6.761986,6.924086,7.086186,7.248286,7.410386,7.572486,...,11.138686,11.300786,11.462886,11.624986,11.787086,11.949186,12.111286,12.273386,12.435486,12.597586
